In [9]:
setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

library(dplyr)
library(data.table)

source("/mnt/lareaulab/reliscu/code/FindModules/FindModules.R")

Allowing multi-threading with up to 40 threads.


Here I run FM to (hopefully) find modules representing each of the cell types present in the single-cell data

In [ ]:
counts <- fread("data/SyntheticDatasets/SyntheticDataset1_20pcntCells_30pcntVar_200samples_10-04-30.csv", data.table=FALSE)

# For duplicate genes, choose row with highest mean expression

mean_expr <- data.frame(
    Index=1:nrow(counts), 
    Gene=counts[,1], 
    Expr=rowMeans(counts[,-1])
)

mean_expr <- mean_expr %>%
    group_by(Gene) %>%
    slice_max(Expr)

print(dim(mean_expr))

counts <- counts[mean_expr$Index,]

[1] 40301     3


In [11]:
# Subset to genes in the top X percentile of mean expression

prob <- .3
mean_expr <- rowMeans(counts[,-1])
print(sum(mean_expr >= quantile(mean_expr, prob)))

[1] 28211


In [12]:
counts_filtered <- counts[mean_expr >= quantile(mean_expr, .5),]

# Normalize counts by sample

total_expr <- colSums(counts_filtered[,-1])
expr <- sweep(counts_filtered[,-1], MARGIN=2, FUN="/", STATS=total_expr) * 1e4
expr <- log2(expr + 1)
expr <- data.frame(Gene=counts_filtered[,1], expr)

In [ ]:
merge.param <- 0.9

In [ ]:
FindModules(
  projectname="tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk",
  expr=expr,
  geneinfo=1,
  sampleindex=2:ncol(expr),
  samplegroups=NULL,
  subset=NULL,
  simMat=NULL,
  saveSimMat=FALSE,
  simType="Bicor",
  beta=1,
  overlapType="None",
  TOtype="signed",
  TOdenom="min",
  MIestimator="mi.mm",
  MIdisc="equalfreq",
  signumType="rel",
  iterate=TRUE,
  signumvec=c(.99999,.9999,.999,.99,.98,.97,.96),
  minsizevec=c(3,4,5,6,8,10),
  signum=NULL,
  minSize=NULL,
  merge.by="ME",
  merge.param=merge.param,
  export.merge.comp=T,
  ZNCcut=2,
  calcSW=FALSE,
  loadTree=FALSE,
  writeKME=TRUE,
  calcBigModStat=FALSE,
  writeModSnap=TRUE
)